In [1]:
import csv

import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from tqdm.notebook import tqdm, trange
from transformers import BertForMaskedLM, BertTokenizer

tqdm.pandas()

In [2]:
matches = pd.read_csv("../../out/matches/matches_dep_constr.tsv", sep="\t", header=0)
matches

,constr,fuzziness (common dep),fuzziness (matches),sentence
0,12,0.062500,0.857143,"Dieser hat allerdings weder die Möglichkeit , ..."
1,12,0.062500,0.857143,Das gestrige Ereignis hatte weder verletzte Pe...
2,15,0.066667,0.857143,Welch ein Image-Gewinn ...
3,15,0.066667,0.857143,"Welch Wunder , dass die Macher nicht noch ein ..."
4,15,0.066667,0.714286,"Welch schönes Gefühl ist es , in Bobotemp-Shor..."
...,...,...,...,...
353444,1986,0.176471,0.800000,"Da eine Bestätigung per E-Mail folgte , hatte ..."
353445,1986,0.176471,0.800000,Für die Konkurrenz der Deutschen Telekom wird ...
353446,1986,0.176471,0.850000,"Inlandsgespräche waren um 7,9 Prozent billiger..."
353447,1986,0.176471,0.800000,Ohne Hightech und neue Ideen hat auch das trad...


In [3]:
with open("../../data/pseudowords/annotations.csv", "r") as csv_file:
    data = [row for row in csv.DictReader(csv_file)]
    
kelex_dict = {}
for example in data:
    kees = set()
    for kee in eval(example["kees"]):
        kees |= set(kee.split())
    kelex_dict[int(example["construction_id"])] = kees

kelex_dict

{10: {'denn', 'geschweige'},
 100: {'gleich'},
 1004: {'er'},
 1006: {'st'},
 101: {'wie'},
 1029: {'Mit', 'wem', 'würden'},
 103: {'wie'},
 1033: {'Hauptsache'},
 1034: {'Von', 'zu'},
 1035: {'von', 'zu'},
 104: {'das', 'nämliche'},
 1043: set(),
 1051: set(),
 1054: set(),
 11: {'kaum', 'und', 'wohl'},
 111: {'Umso', 'je'},
 112: set(),
 1126: {'Generation'},
 1134: {'Sowohl', 'als', 'auch'},
 1140: {'und'},
 1162: {'nicht', 'oder'},
 12: {'noch', 'weder'},
 1219: set(),
 122: {'genauso'},
 123: {'ähnlich'},
 125: {'es', 'sei', 'wie'},
 1257: set(),
 127: {'ein', 'einem', 'von'},
 128: {'artig'},
 1289: {'lassen', 'sein'},
 129: {'Art', 'eine'},
 1291: {'Wenn'},
 13: {'Was', 'für'},
 130: {'wie', 'wäre'},
 1300: {'So', 'geht'},
 1301: {'Augenblick', 'Der'},
 1313: {'Es', 'war', 'zum'},
 1315: {'Als', 'ob'},
 1316: {'Raus', 'aus', 'in', 'rein'},
 132: {'gleicht'},
 1320: {'lein'},
 1322: {'chen'},
 1323: {'pur'},
 1324: {'satt'},
 1329: {'jedermanns'},
 133: {'als', 'ob'},
 1337: {'i'

In [4]:
# cf. evaluate_comapp_bert.ipynb

pseudowords = []
for i in range(15):
    pseudowords.append(np.load(f"../../data/pseudowords/bert/pseudowords_comapp_bert_{i*37}_{i*37+37}.npy"))
pseudowords = np.concatenate(pseudowords)

csv_data = []
for i in range(1, 16):
    csv_data.append(pd.read_csv(f"../../data/pseudowords/bert/order_bert_{i}.csv", sep=";", index_col=0, header=None, quotechar="|", names=["order", "label"]))
csv_data = pd.concat(csv_data)

bert_tokens = [d[0] for d in csv_data.values]
bert_tokens

['""Was13',
 '"647',
 '"Wir-äh-spielen-äh-in-der-äh-Champions-League647',
 '(1597',
 '(1600',
 '(1602',
 '(1624',
 '(1637',
 '(1639',
 '(1641',
 '(1643',
 '(1645',
 '(379',
 '(579',
 '(581',
 '(584',
 '(590',
 '(592',
 '(600',
 '(886',
 '(889',
 '(892',
 '(900',
 '(905',
 '(907',
 '(909',
 '(911',
 '(917',
 '(919',
 '(921',
 '(923',
 ')1597',
 ')1600',
 ')1602',
 ')1624',
 ')1637',
 ')1639',
 ')1641',
 ')1643',
 ')1645',
 ')1792',
 ')379',
 ')579',
 ')581',
 ')584',
 ')590',
 ')592',
 ')600',
 ')886',
 ')889',
 ')892',
 ')900',
 ')905',
 ')907',
 ')909',
 ')911',
 ')917',
 ')919',
 ')921',
 ')923',
 ')«579',
 ',1459',
 ',973',
 '-128',
 '-651',
 '-654',
 '-875',
 '-973',
 ':595',
 ':875',
 ':973',
 'Abstand683',
 'Allein20',
 'Aller1630',
 'Als1315',
 'Als133',
 'Als1770',
 'Am488',
 'Am492',
 'Am500',
 'Amerika605',
 'Anstatt320',
 'Art129',
 'Arzt1509',
 'Augenblick1301',
 'Ausmaß1777',
 'BRUTAL1503',
 'Besser1762',
 'Bis559',
 'Brutal1503',
 'Buche1346',
 'Das1313',
 'Das1461',
 'Da

In [5]:
model = BertForMaskedLM.from_pretrained('bert-base-german-cased', return_dict=True)
tokenizer = BertTokenizer.from_pretrained('bert-base-german-cased')

combined_embeddings = torch.cat((model.bert.embeddings.word_embeddings.weight, torch.tensor(pseudowords)), dim=0)
model.bert.embeddings.word_embeddings = torch.nn.Embedding.from_pretrained(combined_embeddings)
model.bert.embeddings.word_embeddings

Some weights of the model checkpoint at bert-base-german-cased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'cls.seq_relationship.weight', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Embedding(30555, 768)

In [6]:
tokenizer.add_tokens(bert_tokens)
model.resize_token_embeddings(len(tokenizer))

You are resizing the embedding layer without providing a `pad_to_multiple_of` parameter. This means that the new embedding dimension will be 30555. This might induce some performance reduction as *Tensor Cores* will not be available. For more details about this, or help on choosing the correct value for resizing, refer to this guide: https://docs.nvidia.com/deeplearning/performance/dl-performance-matrix-multiplication/index.html#requirements-tc


Embedding(30555, 768)

After everything has been prepared, we can create a list of the contextual embeddings for the standard BERT tokens and the pseudoword tokens.

In [7]:
contextual_embeds = {}

for constr, kelexes in tqdm(kelex_dict.items()):
    contextual_embeds[constr] = {}
    for kelex in kelexes:
        if kelex + str(constr) in bert_tokens:
            original_ids = tokenizer(kelex, return_tensors='pt')['input_ids']
            pseudoword_ids = tokenizer(kelex + str(constr), return_tensors='pt')['input_ids']
            with torch.no_grad():
                original_outputs = model(original_ids, output_hidden_states=True)
                pseudoword_outputs = model(pseudoword_ids, output_hidden_states=True)
            original_contextual_embed = original_outputs.hidden_states[12][0][1:-1]
            pseudoword_contextual_embed = pseudoword_outputs.hidden_states[12][0][1:-1]
            contextual_embeds[constr][kelex] = (original_contextual_embed, pseudoword_contextual_embed)
        else:
            print(kelex + str(constr))

contextual_embeds.keys()

  0%|          | 0/210 [00:00<?, ?it/s]

st1006
würden1029
zu1034
zu1035
ähnlich123
zum1313
lein1320
chen1322
i1337
zu1347
ähnlich139
eln1514
lich1521
leinchen1524
vor1574
Muster1594
ding1621
zum1630
kram1660
Standard1684
keineswegs1690
un1690
en1756
ge1756
ling1842
er381
würdest392
en392
häufigst498
sten500
et571
st623
st631
st65
sau653
en696
über85


dict_keys([10, 100, 1004, 1006, 101, 1029, 103, 1033, 1034, 1035, 104, 1043, 1051, 1054, 11, 111, 112, 1126, 1134, 1140, 1162, 12, 1219, 122, 123, 125, 1257, 127, 128, 1289, 129, 1291, 13, 130, 1300, 1301, 1313, 1315, 1316, 132, 1320, 1322, 1323, 1324, 1329, 133, 1337, 1342, 1346, 1347, 135, 1351, 136, 139, 14, 1459, 1461, 1462, 15, 1503, 1509, 1511, 1514, 1517, 1521, 1524, 1525, 1554, 1556, 1573, 1574, 1582, 1593, 1594, 1597, 16, 1600, 1602, 1621, 1624, 1629, 1630, 1631, 1634, 1636, 1637, 1639, 1641, 1643, 1645, 1649, 1660, 1671, 1681, 1684, 1690, 1715, 1738, 1756, 1760, 1762, 1770, 1772, 1777, 1779, 1792, 1802, 1831, 1835, 1842, 1846, 1849, 1881, 1884, 19, 1986, 1987, 20, 21, 22, 317, 318, 320, 349, 350, 374, 379, 381, 392, 488, 492, 498, 5, 500, 557, 559, 563, 571, 572, 579, 581, 584, 590, 592, 595, 600, 605, 618, 620, 623, 626, 628, 631, 639, 647, 65, 651, 653, 654, 663, 671, 674, 675, 676, 677, 681, 682, 683, 69, 696, 697, 74, 742, 756, 757, 758, 762, 768, 777, 778, 78, 83, 839, 8

In [8]:
contextual_embeds[10]["geschweige"]

(tensor([[-0.1567, -0.2954,  1.1641,  ..., -0.5217,  0.1245, -0.9498],
         [-0.2695, -0.1156,  0.7473,  ...,  0.0997, -0.4645, -0.5735]]),
 tensor([[-5.9337e-01,  1.8925e-01,  8.8151e-01,  7.6941e-01, -1.0799e-01,
          -5.4874e-02, -1.1751e+00, -7.0002e-01, -1.0662e-01,  3.7848e-01,
          -2.2536e-01,  4.8179e-01,  4.9365e-01, -5.8546e-01,  1.5485e-01,
           3.4184e-01,  2.3076e-01, -8.4451e-02, -1.3702e-01, -1.7958e-01,
          -3.6692e-01, -5.9974e-01, -2.9299e-01, -2.8951e-01, -1.0670e+00,
           2.8323e-01, -8.1848e-01, -8.2333e-01,  2.2542e-01,  1.1402e-01,
           9.8367e-01, -3.2775e-01, -1.6153e-02,  4.4972e-01, -4.7634e-01,
           3.5021e-01,  4.6307e-01,  8.4411e-02,  9.5915e-01, -8.9424e-01,
           8.2238e-01,  4.6471e-01, -1.5551e-01, -2.2514e-01,  8.2434e-01,
           2.2404e-01, -6.1623e-01,  4.6392e-01, -2.3587e-02,  4.7538e-01,
          -9.8099e-01,  2.8590e-01,  4.8032e-01, -2.5809e-01, -1.6818e-01,
           4.9187e-01, -3.0922e

Now we can compare the distance of the pseudowords to a sentence with the distance of the original token(s) to the same sentence. Since one pseudoword token can be equivalent to multiple original tokens, we need to take the average distance from all original tokens to compare this average distance to the distance of the pseudoword token.

In [ ]:
sentence_slices_cache = (None, None, None)  # cache of sliced sentences

def distances(row):
    global contextual_embeds
    global sentence_slices_cache
    constr = row["constr"]
    sentence = row["sentence"]
    
    return_row = []
    sentence_ids = tokenizer(sentence, return_tensors='pt')['input_ids']
    # first, check whether the construction has ke-lex and if any ke-lex is in the sentence:
    if (constr not in contextual_embeds.keys()) or (not any([kelex in sentence for kelex in contextual_embeds[constr].keys()])):
        print(".", end="")
        return pd.Series({'constr': row['constr'], 'kelex': None, 'sentence': row['sentence'], 'bert_sim': None, 'pseudword_sim': None, 'bert_euclidean': None, 'pseudword_euclidean': None, 'bert_manhattan': None, 'pseudword_manhattan': None})
    with torch.no_grad():
        if sentence_ids.size(-1) > 512:
            if sentence_slices_cache[0] == sentence:
                _, sentence_id_list, outputs_list = sentence_slices_cache  # load from cache
            else:
                # sliding window approach
                sentence_slices_cache = (None, None, None)  # release cache
                sentence_id_list = [sentence_ids[:, i:i + 512] for i in range(0, sentence_ids.size(-1)-512+1)]
                outputs_list = [model(sentence_ids[:, i:i + 512], output_hidden_states=True) for i in trange(0, sentence_ids.size(-1)-512+1, desc="Slicing")]
                sentence_slices_cache = (sentence, sentence_id_list, outputs_list)
        else:
            sentence_id_list = [sentence_ids]
            outputs_list = [model(sentence_ids, output_hidden_states=True)]
    
        for kelex, embeds in contextual_embeds[constr].items():
            if kelex not in sentence:
                continue
            bert_sims = []
            pseudoword_sims = []
            bert_euclideans = []
            pseudoword_euclideans = []
            bert_manhattans = []
            pseudoword_manhattans = []
            for cur_sentence_ids, outputs in zip(sentence_id_list, outputs_list):
                kelex_ids = [idx for idx, t in enumerate(cur_sentence_ids[0]) if t in tokenizer(kelex, return_tensors='pt')['input_ids'][0][1:-1]]
                if len(kelex_ids) == 0:  # the KE-LEX is not in the current segment
                    continue
                sentence_contextual_embeds = outputs.hidden_states[12][0][kelex_ids]
                
                # Now let's compare BERT and pseudoword:
                bert_sims.append(torch.mean(F.cosine_similarity(embeds[0], sentence_contextual_embeds, dim=-1)))
                pseudoword_sims.append(torch.mean(F.cosine_similarity(embeds[1].expand_as(sentence_contextual_embeds), sentence_contextual_embeds, dim=-1)))
                bert_euclideans.append(torch.mean(torch.norm(embeds[0]-sentence_contextual_embeds, p=2, dim=-1)))
                pseudoword_euclideans.append(torch.mean(torch.norm(embeds[1].expand_as(sentence_contextual_embeds) - sentence_contextual_embeds, p=2, dim=-1)))
                bert_manhattans.append(torch.mean(torch.norm(embeds[0]-sentence_contextual_embeds, p=1, dim=-1)))
                pseudoword_manhattans.append(torch.mean(torch.norm(embeds[1].expand_as(sentence_contextual_embeds) - sentence_contextual_embeds, p=1, dim=-1)))
                
            bert_sim = torch.mean(torch.tensor(bert_sims))
            pseudoword_sim = torch.mean(torch.tensor(pseudoword_sims))
            bert_euclidean = torch.mean(torch.tensor(bert_euclideans))
            pseudoword_euclidean = torch.mean(torch.tensor(pseudoword_euclideans))
            bert_manhattan = torch.mean(torch.tensor(bert_manhattans))
            pseudoword_manhattan = torch.mean(torch.tensor(pseudoword_manhattans))
            return_row.append({'constr': row['constr'], 'kelex': kelex, 'sentence': row['sentence'], 'bert_sim': float(bert_sim), 'pseudword_sim': float(pseudoword_sim), 'bert_euclidean': float(bert_euclidean), 'pseudword_euclidean': float(pseudoword_euclidean), 'bert_manhattan': float(bert_manhattan), 'pseudword_manhattan': float(pseudoword_manhattan)})
            
            if any([pseudoword_sim >= bert_sim, pseudoword_euclidean <= bert_euclidean, pseudoword_manhattan <= bert_manhattan]):
                print("\n" + str(return_row))
            else:
                print(":", end="")
        return pd.Series(return_row)

similarities = matches.progress_apply(distances, axis=1)
similarities

  0%|          | 0/353449 [00:00<?, ?it/s]

::
[{'constr': 12, 'kelex': 'noch', 'sentence': 'Das gestrige Ereignis hatte weder verletzte Personen zur Folge , noch kam die Produktionsstätte von Texas Instruments durch das Feuer direkt zu Schaden .', 'bert_sim': 0.46541234850883484, 'pseudword_sim': 0.4611941874027252, 'bert_euclidean': 23.424827575683594, 'pseudword_euclidean': 23.35661506652832, 'bert_manhattan': 505.57733154296875, 'pseudword_manhattan': 498.20782470703125}]
:...::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::...:::.:::::::.......::
[{'constr': 100, 'kelex': 'gleich', 'sentence': ' Wir werden nicht allen 4,5 Millionen Teilnehmern gleich eine neue CD schicken  , so Kleuntje .', 'bert_sim': 0.679663360118866, 'pseudword_sim': 0.70242840051651, 'bert_eucl

In [ ]:
similarities.to_csv(f"../../out/comapp/similarities_bert.tsv", sep="\t", decimal=",")
similarities.to_excel(f"../../out/comapp/similarities_bert.xlsx")